In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
path=r"C:\Users\HP\churn pipeline\train_data\WA_Fn-UseC_-Telco-Customer-Churn.csv"
data=pd.read_csv(path)
data

In [ ]:
non_unique = data["customerID"].duplicated(keep=False)
non_unique.unique() #no repeated columns

In [ ]:
every_columns=data.columns
data["OnlineBackup"] #catigorical variable
every_columns

In [ ]:
categorical_data=pd.concat([data["gender"],data["SeniorCitizen"],data["Partner"],
             data["Dependents"],data["tenure"],data["PhoneService"],data["MultipleLines"],data["InternetService"],
             data["OnlineSecurity"],data["OnlineBackup"],data['DeviceProtection'],data['TechSupport'],data['StreamingTV'], 
             data["StreamingMovies"],data["Contract"],data["PaperlessBilling"],data['PaymentMethod'],data['Churn']]
                         ,axis=1).astype('category', copy=False)
categorical_data

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
encoded_data = categorical_data.apply(lambda col: le.fit_transform(col))
encoded_data

In [ ]:
concat_data=pd.concat([data['MonthlyCharges'],data['TotalCharges']],axis=1)
filltered_data=pd.concat([concat_data,encoded_data],axis=1)
filltered_data.isnull().sum()

In [ ]:
for col in filltered_data.columns:
    num_spaces = (filltered_data[col] == ' ').sum()
    num_empty = (filltered_data[col] == '').sum()
    if num_spaces > 0 or num_empty > 0:
        print(f"Column '{col}' has {num_spaces} spaces and {num_empty} empty strings")

In [ ]:
mask = filltered_data.applymap(lambda x: isinstance(x, str) and x.strip() == '')
filltered_data = filltered_data[~mask.any(axis=1)]

In [ ]:
filltered_data

In [ ]:
fig,ax=plt.subplots(5,4,figsize=(12,12)) #filltered_data
counts=0
l=[]
for j in range(5):
    for c in range(4):
        
        col = filltered_data.columns[counts]
        ax[j][c].hist(filltered_data[col], bins=20, color='skyblue', edgecolor='black')
        ax[j][c].set_title(f'Distribution of {col}')
        counts += 1

ax[j][c].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
filltered_data

In [ ]:
corr_matrix = filltered_data.corr(method='spearman')
corr_matrix

In [ ]:
sns.countplot(data=filltered_data,x="Churn")

In [16]:
import os

save_dir = '../data/processed'
save_path = os.path.join(save_dir,'filltered_data.csv')

os.makedirs(save_dir, exist_ok=True)
filltered_data.to_csv(save_path, index=False)

In [17]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

iso_forest = IsolationForest(contamination=0.1)
iso_pred = iso_forest.fit_predict(filltered_data)

# Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
lof_pred = lof.fit_predict(filltered_data)

# One-Class SVM
ocsvm = OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
ocsvm.fit(filltered_data)
svm_pred = ocsvm.predict(filltered_data)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

methods = ['Isolation Forest', 'Local Outlier Factor', 'One-Class SVM']
predictions = [iso_pred, lof_pred, svm_pred]

for i, ax in enumerate(axs):
    
    ax.set_title(methods[i])
    inliers = filltered_data[predictions[i] == 1]
    outliers = filltered_data[predictions[i] == -1]
    
    ax.scatter(inliers.iloc[:, 0], inliers.iloc[:, 1], c='blue', label='Inliers')
    ax.scatter(outliers.iloc[:, 0], outliers.iloc[:, 1], c='red', label='Outliers')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
mask = iso_pred == 1
X_cleaned = filltered_data[mask]
type(X_cleaned)

In [20]:
from imblearn.over_sampling import SMOTE

y=X_cleaned["Churn"]
x=X_cleaned.drop("Churn",axis=1)

X_resampled,y_resampled =SMOTE().fit_resample(x, y)

In [ ]:
y_resampled.shape

In [ ]:

c=pd.concat([X_resampled,y_resampled],axis=1)
c

In [ ]:
df=pd.DataFrame(y_resampled)
sns.countplot(data=df,x="Churn")

In [ ]:
X_resampled.shape,y_resampled.shape

In [23]:
y_resampled=pd.DataFrame(y_resampled)
proced_data=pd.concat([X_resampled,y_resampled])

In [24]:
import os

save_dir = '../data/processed'
save_path = os.path.join(save_dir,'processed_data.csv')

os.makedirs(save_dir, exist_ok=True)
proced_data.to_csv(save_path, index=False)

In [ ]:
proced_data.iloc[:,:-1]